In [1]:
import matplotlib.pyplot as plt
import random, shutil, pickle, sys
import numpy as np
from collections import deque
from keras import Sequential
from keras.optimizers import RMSprop
from IPython.display import display
from PIL import Image
import pandas as pd
from keras import Model
import keras, os, glob
import tensorflow as tf
from keras.layers import Layer, Dense, Conv2D, Flatten, RepeatVector,Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras.layers import Activation, LSTM, Bidirectional , Dropout
from keras import layers
# import cv2
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import codecs
import csv
import secrets
import sqlite3
from IPython.display import display
from IPython.display import clear_output
import time
from tabulate import tabulate
import json
import datetime
from tensorflow.keras.optimizers import Adam

Recommedations. add account balance on the chart the current trade its profit closing profit 

also add different consipicous icons for
opening the trade
holding the trade in profit
holding the trade in loss
show reversal arrows for a trade that was in profit and now it is in loss
show reversal arrows for a trade that was in loss and now is in profit give a reward of 0.6 and holding a profitable trade 0.8 

use cases for rewards

first practise drawing on the chart ask chart gpt for the lines


Rewards
0 == idle = 0.2
0 == close == profit ==  1
0 == close == loss   == -1

1 == open == 0.4
1 == close == profit == 0.9
1 == close == loss == -0.9
1 == hold == profit == 0.7
1 == hold == loss == -0.7

2 == open == 0.4
2 == close == profit == 0.9
2 == close == loss == -0.9
2 == hold == profit == 0.7
2 == hold == loss == -0.7

use case 2
a trade that was in loss and now its in profit 
1 == hold == loss == profit == 0.8
2 == hold == loss == profit == 0.8

use case 3
a trade that was in profit and now its in loss
1 == hold == loss == profit == -0.8
2 == hold == loss == profit == -0.8

use case 2 and 3 need a bar that when it reverses the the bar widens to accomodate the change of a reversing arrow show strange graffiti on the bar


use case 4
account balance show
:profit
:accumulative profit
:current_trade

use case to achieve above objectives we need to have a history of the current trade called 
current_history()
:highest _ price
:lowest _ price

use case 7:
when the price goes really high above a certain point we need a different animation on the chart
so that it can know that its really time to be ready for the next time

This usecase commands that the change in price for the algorithm the rise of profit in the chart should be consipicous
such that it reflects different bars

1:/:2 == profit == 10 different bar
1:/:2 == loss == -10 different bar

1:/:2 == profit == 20 different bar
1:/:2 == loss == -20 different bar

1:/:2 == profit == 30 different bar
1:/:2 == loss == -30 different bar

1:/:2 == profit == 40 different bar
1:/:2 == loss == -40 different bar


1:/:2 == profit == 50 different bar
1:/:2 == loss == -50 different bar


1:/:2 == profit == 60 different bar
1:/:2 == loss == -60 different bar


1:/:2 == profit == 70 different bar
1:/:2 == loss == -70 different bar

1:/:2 == profit == 80 different bar
1:/:2 == loss == -80 different bar

1:/:2 == profit == 90 different bar
1:/:2 == loss == -90 different bar

1:/:2 == profit >= 100 different bar
1:/:2 == loss == 100 different bar


use case 8 

create better entry colors on the chart

the empty parts of the chart should include the effects that are happening on the forex environment

use case 9

the time steps should have the most recent image so that the missing aspect of loss presented after a timestep can be eliminated

so the next model with have a shape of (1,5,224,224,3) the fifth frame should have the most recent time step for a step of 1

frame1 = fifth image  =  0   += 1 = 1
frame2 = fourth image =  32  += 1 = 33
frame3 = third image  =  64  += 1 = 65
frame4 = second image =  128 += 1 = 129
frame5 = first image  =  129 += 1 = 130



#Define Agent

In [2]:
m5long = pd.read_csv("final_m5long.csv")
m5short = pd.read_csv("final_m5short.csv")
m1long = pd.read_csv("final_m1long.csv")
m1short = pd.read_csv("final_m1short.csv")

In [3]:
# def display_row_images(long_short_dict_now , long_short_dict_next):
from collections import deque
class ForexCustomEnv:
    def __init__(self, m1short):
        self.m1short = m1short
        self.trade_queue = deque(maxlen=4)
        self.image_queue = deque(maxlen=4)
        self.dataset_directory = "colabM1M5/episode2/"
        self.account_balance = 1000

    def get_ask_bid_lines(self, image_path):
        ask_bid_lines = {"ask_line":None, "bid_line":None, "key": 0}
        img = Image.open(image_path)
        img_data = np.array(img)
        last_part = Image.fromarray(img_data[:,200:201,:])
        img1_pixels = last_part.load()
        ask_color = (255,0,0)
        bid_color = (119, 136, 153)
        for y in range(224):
            if img1_pixels[0,y] == ask_color:
                ask_bid_lines["ask_line"] = y
            if img1_pixels[0,y] == bid_color:
                ask_bid_lines["bid_line"] = y
                
        return ask_bid_lines , img          
    
    def reset(self):
        self.current_step = 4
        linesq, imageq = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[self.current_step-4]['image_path']}")
        linesq["key"] = 0
        linest, imaget = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[self.current_step-3]['image_path']}")
        linest["key"] = 1
        liness, images = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[self.current_step-2]['image_path']}")
        liness["key"] = 2
        linesc, imagec = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[self.current_step-1]['image_path']}")
        linesc["key"] = 3
        self.trade_queue.append(linesq)
        self.trade_queue.append(linest)
        self.trade_queue.append(liness)
        self.trade_queue.append(linesc)
        self.image_queue.append(imageq)
        self.image_queue.append(imaget)
        self.image_queue.append(images)
        self.image_queue.append(imagec)    
        self.done = False
        self.reward = 0
        self.position = None
        self.reset_current_trade()
        self.account_balance = 1000
        next_state = self.get_obs()
        
        return next_state, self.reward, self.done, self.current_trade 
        
    def reset_current_trade(self):
        self.current_trade = {}
        self.current_trade["current_step"] = self.current_step
        self.current_trade["ask_bid_lines"] = None
        self.current_trade["entry_price"] = None
        self.current_trade["current_price"] = None
        self.current_trade["timesteps"] = 0
        self.current_trade["position"] = None
        self.current_trade["profit"] = 0
        self.current_trade["position"] = None
        self.current_trade["parent_ask_bid_lines"] = None
        self.current_trade["reward"] = 0
        self.current_trade["balance"] = self.account_balance
        
    def display_stacked_horizontally(self,imageq, imaget,images,imagec):
        # # Calculate the required dimensions for the new image
        new_width = imageq.width * 4
        new_height = imageq.height
        # # Create a new image with the required dimensions
        new_image = Image.new("RGB", (new_width, new_height))
        # # Paste the individual images side by side
        new_image.paste(imageq, (0, 0))
        new_image.paste(imaget, (imageq.width, 0))
        new_image.paste(images, (imageq.width + imaget.width, 0))
        new_image.paste(imagec, (imageq.width + imaget.width + images.width, 0))
        

    
    def reset_collection(self, t_q, i_q):
        _queue = deque(maxlen=4)
        for i in range(0, 4):
            _lines_c, _image_c = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[t_q[i]['key']]['image_path']}")
            _image_c = self.draw_buy_bar(_image_c, _lines_c, _lines_c, position="close", profit = 0)
            _queue.append(_image_c)
        return t_q, _queue
    
    def draw_collection(self,q):
        # # Calculate the required dimensions for the new image
        new_width = q[0].width * 4
        new_height =q[0].height
        # # Create a new image with the required dimensions
        new_image = Image.new("RGB", (new_width, new_height))
        # # Paste the individual images side by side
        new_image.paste(q[0], (0, 0))
        new_image.paste(q[1], (q[0].width, 0))
        new_image.paste(q[2], (q[0].width + q[0].width, 0))
        new_image.paste(q[3], (q[0].width + q[0].width + q[0].width, 0))
        display(new_image)
        
    def draw_buy_bar(self, current_image, start_lines, end_lines, position = None, profit = 0):
        try:
            if start_lines["ask_line"] == None:
                start_lines["ask_line"] = start_lines["bid_line"] + 2
            elif start_lines["bid_line"] == None:
                start_lines["bid_line"] == start_lines["ask_line"] - 2
            elif end_lines["ask_line"] == None:
                end_lines["ask_line"] = end_lines["bid_line"] + 2
            elif end_lines["bid_line"] == None:
                end_lines["bid_line"] == end_lines["ask_line"] - 2
                
            from PIL import Image, ImageDraw
            d = ImageDraw.Draw(current_image)
            if self.position == "sell":
                # Draw indicator of trade start
                line_color = (200,200,200)
                top = (200, end_lines["ask_line"])
                bottom = (200, end_lines["bid_line"])

                d.line([top, bottom], fill=line_color, width=20)
            #Draw profit indicator of sell
                if profit < 0:
                    line_color = (0, 0, 255)
                    top = (200, start_lines["ask_line"])
                    bottom = (200,end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=10)
                else:
                    line_color = (0, 255, 0)
                    top = (200, start_lines["ask_line"])
                    bottom = (200,end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=10)
            #Draw loss indicator for sell
            elif self.position == "buy":
                #Draw indicator of trade start
                line_color = (0,255,0)
                top = (200, end_lines["ask_line"])
                bottom = (200, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=20)        
                if profit < 0:
                    line_color = (255, 0, 0)
                    top = (200, start_lines["ask_line"])
                    bottom = (200, end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=10)
                else:
                    line_color = (255, 255, 0)
                    top = (200, start_lines["ask_line"])
                    bottom = (200, end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=10)
            else:
                line_color = (255,0,255)
                top = (200, start_lines["ask_line"])
                bottom = (200, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=40)
        except Exception as e:
            print(f"\n\n\nstart_lines: {start_lines} end_lines:{end_lines}{e}\n\n\n")
        return current_image
    
    '''
    @description: Enters a trade in the forex environment
                  Enters the trade in the current time step, sets the position of the trade. sets the ask bid lines for the current frame
                  sets the entry price for the trade, and profit = 0. Then draws on the image the entry icon on the chart
                  and saves the updated current_image with modifications to the image queue with its corresponding ask and bid lines. 
                  then initializes the timesteps of the current trade to 0. sets the reward to 0.3
    @params
                  @position = None the position of the current_trade
                  @ask_bid_lines of the current_image
                  @current_image the current frame of the environment
                  
    '''
    
    def enter_trade(self, ask_bid_lines,current_image):
        self.current_trade["position"] = self.position
        self.current_trade["current_step"] = self.current_step
        self.current_trade["ask_bid_lines"] = ask_bid_lines
        self.current_trade["entry_price"] = float(f"{self.m1short.iloc[self.current_step]['Ask']}") if self.position == "buy" else float(f"{self.m1short.iloc[self.current_step]['Bid']}")
        self.current_trade["profit"] = 0
        self.current_trade["current_price"] = float(f"{self.m1short.iloc[self.current_step]['Ask']}")
        self.current_trade["timesteps"] = 0
        self.current_trade["reward"] = 0.4
        self.current_trade["parent_ask_bid_lines"] = ask_bid_lines
        self.current_trade["balance"] = self.account_balance
        current_image = self.draw_buy_bar(current_image, ask_bid_lines, ask_bid_lines, self.position , self.current_trade["profit"])
        self.trade_queue.append(ask_bid_lines)
        self.image_queue.append(current_image)  
        #self.draw_collection(self.image_queue)
        self.reward = 0.4
        
    def enter_idle_mode(self, ask_bid_lines, current_image ):
        self.current_trade["current_step"] = self.current_step
        self.current_trade["ask_bid_lines"] = ask_bid_lines
        self.current_trade["entry_price"] = float(f"{self.m1short.iloc[self.current_step]['Ask']}")
        self.current_trade["current_price"] = float(f"{self.m1short.iloc[self.current_step]['Ask']}")
        self.current_trade["position"] = self.position
        self.current_trade["reward"] = 0.2
        self.current_trade["timesteps"] += 1
        self.current_trade["profit"] = 0
        self.current_trade["parent_ask_bid_lines"] = None
        self.current_trade["balance"] = self.account_balance
        current_image = self.draw_buy_bar(current_image, ask_bid_lines, ask_bid_lines, self.position, self.current_trade["profit"])
        self.trade_queue.append(ask_bid_lines)
        self.image_queue.append(current_image)  
        #self.draw_collection(self.image_queue)            
        self.reward = 0.2 
        
    def hold_position(self,ask_bid_lines, current_image):
        
        self.current_trade["parent_ask_bid_lines"] = self.parent_ask_bid_lines
        self.current_trade["current_price"] = float(f"{self.m1short.iloc[self.current_step]['Bid']}") if self.position == "buy" else float(f"{self.m1short.iloc[self.current_step]['Ask']}")
        self.current_trade["profit"] = self.current_trade["current_price"] - self.current_trade["entry_price"] if self.position == "buy" else self.current_trade["entry_price"] - self.current_trade["current_price"]
        self.current_trade["timesteps"] += 1
        self.current_trade["balance"] = self.account_balance
        current_image = self.draw_buy_bar(current_image, self.current_trade["parent_ask_bid_lines"], ask_bid_lines, self.position , self.current_trade["profit"])
        self.trade_queue.append(ask_bid_lines)
        self.image_queue.append(current_image)
        #self.draw_collection(self.image_queue)
        if self.current_trade["profit"] >= 0:
            self.reward = 0.6
            self.current_trade["reward"] = self.reward
        else:
            self.reward = -0.6
            self.current_trade["reward"] = self.reward

        
                
    def close_position(self, ask_bid_lines, current_image):
        
        self.current_trade["current_price"] = float(f"{self.m1short.iloc[self.current_step]['Bid']}") if self.position == "buy" else float(f"{self.m1short.iloc[self.current_step]['Ask']}")
        self.current_trade["profit"] = self.current_trade["current_price"] - self.current_trade["entry_price"] if self.position == "buy" else self.current_trade["entry_price"] - self.current_trade["current_price"]
        current_image = self.draw_buy_bar(current_image, self.parent_ask_bid_lines, ask_bid_lines, self.position, self.current_trade["profit"] )
        self.trade_queue.append(ask_bid_lines)
        self.image_queue.append(current_image)
        #self.draw_collection(self.image_queue)
        if self.current_trade["profit"] >= 0:
            self.reward = 1
        else:
            self.reward = -1          
        self.trade_queue, self.image_queue = self.reset_collection(self.trade_queue,self.image_queue)
        self.account_balance += self.current_trade["profit"]
        self.current_trade["current_timestep"] = self.current_step
        self.current_trade["timesteps"] += 1
        self.current_trade["position"] = None
        self.current_trade["balance"] = self.account_balance
        self.position = None
        self.parent_ask_bid = None
        
    def step(self, action):
        self.reward = 0
        self.done = False
        ask_bid_lines, current_image = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[self.current_step]['image_path']}")
        ask_bid_lines["key"] = self.current_step
        if self.position == None:
            if action == 1:
                self.position = "buy"
                self.parent_ask_bid_lines = ask_bid_lines
                self.enter_trade(ask_bid_lines,current_image)
            elif action == 2:
                self.position = "sell"
                self.parent_ask_bid_lines = ask_bid_lines
                self.enter_trade(ask_bid_lines,current_image)
            elif action == 0:
                self.position = None
                self.parent_ask_bid_lines = ask_bid_lines
                self.enter_idle_mode(ask_bid_lines, current_image)
        elif self.position == "buy":
            if action == 1:
                self.hold_position(ask_bid_lines, current_image)
            elif action in [0,2]:
                self.close_position(ask_bid_lines, current_image)
        elif self.position == "sell":
            if action == 2:
                self.hold_position(ask_bid_lines, current_image)
            elif action in [1,0]:
                self.close_position(ask_bid_lines, current_image)
                self.position = None
        self.current_step+=1
        next_state = self.get_obs()
        if self.account_balance < 920:
            self.done = True
        return next_state, self.reward, self.done, self.current_trade
    
    def get_obs(self):
        stacked_obs = np.expand_dims(np.stack([np.asarray(self.image_queue[0])/255, np.asarray(self.image_queue[1])/255, np.asarray(self.image_queue[2])/255, np.asarray(self.image_queue[3])/255]), axis=0)
        
        return np.expand_dims(stacked_obs, axis=0)


In [4]:
class TimeSformerBlock(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, num_heads):
        super(TimeSformerBlock, self).__init__()
        
        # Define the self-attention layer
        self.self_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_dim)
        
        # Define the feedforward layer
        self.feedforward = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim * 4, activation='relu'),
            tf.keras.layers.Dense(hidden_dim)
        ])
        
        # Define the layer normalization layers
        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()
        
    def call(self, x):
        # Apply layer normalization and self-attention
        norm_x = self.norm1(x)
        attention_output = self.self_attention(norm_x, norm_x)
        x = x + attention_output
        
        # Apply layer normalization and feedforward layer
        norm_x = self.norm2(x)
        feedforward_output = self.feedforward(norm_x)
        x = x + feedforward_output
        
        return x

In [19]:
class Agent:
    def __init__(self, state_size, action_size, env, learning_rate=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.gamma = 0.001
        self.env = env
        self.actor = self._build_model()
        self.critic = self._build_model()
        self.critic.set_weights(self.actor.get_weights())
        self.models_dir = "models/attention/"
        self.actor_model_name = "att_actor.h5"
        self.critic_model_name = "att_critic.h5"
        self.actor.summary()
        
        
    def _build_model(self):
        cnn = Sequential()
        cnn.add(tf.keras.layers.Conv3D(32, (3,3,3), strides=(1,4,4), padding="same"))
        cnn.add(Activation('relu'))
        cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same'))
        cnn.add(tf.keras.layers.Conv3D(64, (3,3,3), padding="same"))
        cnn.add(Activation('relu'))
        cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same'))
        cnn.add(tf.keras.layers.Conv3D(64, (3,3,3), padding="same"))
        cnn.add(Activation('relu'))
        cnn.add(Flatten())
        cnn.add(Dense(256, activation='relu'))
        transformer = TimeSformerBlock( num_heads=8, hidden_dim=256)
        model=Sequential()
        model.add(tf.keras.layers.TimeDistributed(cnn,input_shape=self.state_size))
        model.add(transformer)
        model.add(Dense(32, activation='relu'))
        # model.add(Dense(16, activation='relu'))
        model.add(Dense(self.action_size, activation='softmax'))
        model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=1e-4))
        return model
    
    def predict(self, state):
        return self.actor.predict(state)
        
    def save(self):
        self.actor.save(f"{self.models_dir}{self.actor_model_name}")
        self.critic.save(f"{self.models_dir}{self.critic_model_name}")
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return secrets.randbelow(self.action_size) if np.random.choice(20) % 2 == 0 else np.random.choice(self.action_size) , [[0,0,0]]
        else:
            pred = self.actor.predict(state)
            return np.argmax(pred[0][0]), pred 
        
    def load(self):
        self.actor.load_weights(f"{self.models_dir}{self.actor_model_name}")
        self.critic.load_weights(f"{self.models_dir}{self.critic_model_name}")
        
        
    def train(self, num_episodes):
        header = ["episode", "counter","action", "reward", "profit", "done", "balance", "actor", "critic", "Eprice", "Cprice"]
        self.episode_rewards = []
        target_f_values = []
        for episode in range(num_episodes):
            state, reward, done, _ = self.env.reset()
            total_rewards = 0
            counter = 0
            
            while not done:
                rows = []
                
                action, pred = self.act(state)
                next_state, reward, done, current_trade = self.env.step(action)
                total_rewards += reward
                
                if not done:
                    target_f_values = self.critic.predict(next_state)
                    target_f = reward + self.gamma * np.argmax(target_f_values[0][0])
                else:
                    target_f = reward
                
                q_values = self.actor.predict(state)
                
                row = [episode,counter, action, target_f, current_trade["profit"], done, current_trade["balance"], ",".join(str("{:.2f}".format(x)) for x in list(q_values)[0][0]), ",".join(str("{:.2f}".format(x)) for x in list(target_f_values)[0][0])]
                
                q_values[0][0][action] = target_f
                
                rows.append(row)
                print(tabulate(rows,headers = header, tablefmt="grid"))
                
                self.actor.fit(state, q_values, verbose=1)
                
                state = next_state
                
                if counter % 100 == 0:
                    self.save()
                    self.critic.set_weights(self.actor.get_weights())
                            # Decay the exploration rate
                if self.epsilon > self.epsilon_min and counter % 50 == 0:
                    self.epsilon *= self.epsilon_decay
                if counter > 1200:
                    done = True
                
                counter += 1
            clear_output(wait=True)
            if episode % 5 == 0:
                self.critic.set_weights(self.actor.get_weights())
            if episode % 3 == 0:
                self.save()
            # Decay the exploration rate
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
            self.episode_rewards.append(total_rewards)
            print(self.episode_rewards)
        return self.episode_rewards
    

In [ ]:
if __name__ == "__main__":
    action = 3
    state_size = (1, 4, 224, 224, 3)
    env = ForexCustomEnv(m1short)
    agent = Agent(state_size, action, env)
    agent.epsilon = 0.001
    agent.load()
    agent.env = env
    episode_rewards = agent.train(num_episodes=100)

# import matplotlib.pyplot as plt
# plt.plot(episode_rewards)
# plt.xlabel('Episode')
# plt.ylabel('Total Reward')
# plt.show()

# real work

In [20]:
# agent.epsilon

0.9509900498999999

In [ ]:
ticks = [ 0,0,0,0,1,1,1,1,1,1,1,0,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1, 0, 2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

In [ ]:
# episode_rewards = agent.train(num_episodes=1)
# agent.epsilon = 0.001
# import matplotlib.pyplot as plt
# plt.plot(episode_rewards)
# plt.xlabel('Episode')
# plt.ylabel('Total Reward')
# plt.show()

In [21]:
# agent.save()

In [31]:
# episode_rewards

[59.20000000000001]